In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from utils.matrix_builder import create_expanded_feature_matrix, create_feature_matrix

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load Database
csv_hour_file = '../data/ta_metrics/final_price_ta_metrics.csv'
df = pd.read_csv(csv_hour_file, parse_dates=['Datetime'])

def random_forest_training(df, model, sliding_window, lag_price_window, DEBUG):
    """
    Train a Random Forest model using a sliding window approach.

    Parameters:
    - df: DataFrame containing the dataset with features and target variable.
    - model: The machine learning model to be trained (e.g., RandomForest).
    - sliding_window: Number of rows to use for training in each sliding window.
    - lag_price_window: Number of previous days to use as features.
    
    Returns:
    - prediction_df: DataFrame containing predictions and actual values.
    """
    if DEBUG:
        print("Debug mode is ON. Detailed output will be printed.")

    # Validate input parameters
    if sliding_window <= lag_price_window:
        raise ValueError("Sliding window must be greater than to the price feature window.")

    training_sliding_window = sliding_window + 1  # +1 to include the next row as the test set

    # Calculate number of sliding window models to train in the dataset
    num_sliding_windows = len(df) - training_sliding_window
    if DEBUG:
        # Training for sliding windows and price feature window
        print(f"Training sliding window size: {training_sliding_window}, Price feature window size: {lag_price_window}")
        print(f"Number of rows in the dataset: {len(df)}")

    print(f"Number of models to train: {num_sliding_windows}")

    # Initialize lists to store predictions, actuals, and timestamps
    predictions_list = []
    actuals_list = []
    timestamps_list = []

    model = model  # This is done in order to use different model configurations of RandomForestRegressor

    for i in range(num_sliding_windows):
        if DEBUG:
            print(f"Processing sliding window {i + 1}/{num_sliding_windows}...")

        # Ensure we do not exceed the DataFrame length
        if i + training_sliding_window >= len(df):
            break  # Avoid index out of bounds
        
        sliding_window_set = df.iloc[i : i + training_sliding_window]

        if DEBUG:
            print(f"Sliding window should have {training_sliding_window} rows, got {len(sliding_window_set)} rows.")
            print(f"Sliding window set:\n{sliding_window_set}")
        
        # Create feature matrix and target variable for training
        X_train, y_train = create_expanded_feature_matrix(sliding_window_set, lag_price_window)
        if DEBUG:
            print(f"Feature matrix shape: {X_train.shape}, Target variable shape: {y_train.shape}")
            print(f"Feature matrix:\n{X_train.head()}")
            print(f"Target variable:\n{y_train.head()}")

        # Split for training and prediction
        X_train_fit = X_train.iloc[:-1]
        y_train_fit = y_train.iloc[:-1]

        X_to_predict = X_train.iloc[-1:]
        y_to_predict = y_train.iloc[-1]

        if DEBUG:
            print(f"Training features shape: {X_train_fit.shape}, Training target shape: {y_train_fit.shape}")
            print(f"Features to predict shape: {X_to_predict.shape}, Target to predict: {y_to_predict}")

        # Scale features for better convergence
        scaler = StandardScaler()
        X_train_fit_scaled = scaler.fit_transform(X_train_fit)
        X_to_predict_scaled = scaler.transform(X_to_predict)

        model.fit(X_train_fit_scaled, y_train_fit)
        y_predicted = model.predict(X_to_predict_scaled)
        
        # Add a lower bounds to set extreme negative predictions to 0, assuming prices cannot be negative
        if y_predicted[0] < 0:
            y_predicted[0] = 0

        # Store results
        predictions_list.append(y_predicted[0])
        actuals_list.append(y_to_predict)
        if 'Datetime' in sliding_window_set.columns:
            timestamps_list.append(sliding_window_set.iloc[-1]['Datetime'])
        else:
            timestamps_list.append(i + training_sliding_window - 1)

    # Create final prediction DataFrame
    prediction_df = pd.DataFrame({
        'Timestamp': timestamps_list,
        'Predicted': predictions_list,
        'Actual': actuals_list
    })

    return prediction_df



def random_forest_training_gt(df, model, sliding_window, lag_price_window, DEBUG):
    """
    Train a Random Forest model using a sliding window approach.

    Parameters:
    - df: DataFrame containing the dataset with features and target variable.
    - model: The machine learning model to be trained (e.g., RandomForest).
    - sliding_window: Number of rows to use for training in each sliding window.
    - lag_price_window: Number of previous days to use as features.
    
    Returns:
    - prediction_df: DataFrame containing predictions and actual values.
    """
    if DEBUG:
        print("Debug mode is ON. Detailed output will be printed.")

    # Validate input parameters
    if sliding_window <= lag_price_window:
        raise ValueError("Sliding window must be greater than to the price feature window.")

    training_sliding_window = sliding_window + 1  # +1 to include the next row as the test set

    # Calculate number of sliding window models to train in the dataset
    num_sliding_windows = len(df) - training_sliding_window
    if DEBUG:
        # Training for sliding windows and price feature window
        print(f"Training sliding window size: {training_sliding_window}, Price feature window size: {lag_price_window}")
        print(f"Number of rows in the dataset: {len(df)}")

    print(f"Number of models to train: {num_sliding_windows}")

    # Initialize lists to store predictions, actuals, and timestamps
    predictions_list = []
    actuals_list = []
    timestamps_list = []

    model = model  # This is done in order to use different model configurations of RandomForestRegressor

    for i in range(num_sliding_windows):
        if DEBUG:
            print(f"Processing sliding window {i + 1}/{num_sliding_windows}...")

        # Ensure we do not exceed the DataFrame length
        if i + training_sliding_window >= len(df):
            break  # Avoid index out of bounds
        
        sliding_window_set = df.iloc[i : i + training_sliding_window]

        if DEBUG:
            print(f"Sliding window should have {training_sliding_window} rows, got {len(sliding_window_set)} rows.")
            print(f"Sliding window set:\n{sliding_window_set}")
        
        # Create feature matrix and target variable for training
        X_train, y_train = create_feature_matrix(sliding_window_set, lag_price_window)
        if DEBUG:
            print(f"Feature matrix shape: {X_train.shape}, Target variable shape: {y_train.shape}")
            print(f"Feature matrix:\n{X_train.head()}")
            print(f"Target variable:\n{y_train.head()}")

        # Split for training and prediction
        X_train_fit = X_train.iloc[:-1]
        y_train_fit = y_train.iloc[:-1]

        X_to_predict = X_train.iloc[-1:]
        y_to_predict = y_train.iloc[-1]

        if DEBUG:
            print(f"Training features shape: {X_train_fit.shape}, Training target shape: {y_train_fit.shape}")
            print(f"Features to predict shape: {X_to_predict.shape}, Target to predict: {y_to_predict}")

        model.fit(X_train_fit, y_train_fit)
        y_predicted = model.predict(X_to_predict)
        
        # Add a lower bounds to set extreme negative predictions to 0, assuming prices cannot be negative
        if y_predicted[0] < 0:
            y_predicted[0] = 0

        # Store results
        predictions_list.append(y_predicted[0])
        actuals_list.append(y_to_predict)
        if 'Datetime' in sliding_window_set.columns:
            timestamps_list.append(sliding_window_set.iloc[-1]['Datetime'])
        else:
            timestamps_list.append(i + training_sliding_window - 1)

    # Create final prediction DataFrame
    prediction_df = pd.DataFrame({
        'Timestamp': timestamps_list,
        'Predicted': predictions_list,
        'Actual': actuals_list
    })

    return prediction_df

Baseline (Ground truth)

In [ ]:
# Debug parameter
DEBUG = False

# Define the Sliding Windows for the runs
sliding_windows = [7, 10, 15, 30, 90]  # of days to train on (matrix rows)
lag_price_windows = [1, 2, 4, 6]  # of previous days to use as features (matrix columns)
# lag_price_window must be less than sliding_window
percentiles = [99, 95, 85, 75]

# Random Forest hyperparameters
n_estimators = [100, 200, 300, 400]  # Number of trees in the Random Forest
# Start with a moderately high number (e.g., 100-200) and increase it until the validation error no longer significantly improves.

max_depth = [None, 10, 20, 30]  # Maximum depth of the trees
# While default None often works well for Random Forests due to the ensemble nature
# you might experiment with values like 10, 20, or 30
# to control complexity if you observe overfitting.

# Random Forest model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=42)
# Set n_jobs=-1 to use all available cores for parallel processing
# Set random_state for reproducibility

for n_estimators in n_estimators:
    # Initialize results file path
    results_file = f"random_forest_baseline_results_n_estimators_{n_estimators}.csv"
    
    # Check if results file already exists and load existing results
    try:
        existing_results_df = pd.read_csv(results_file)
        comprehensive_baseline_results = existing_results_df.to_dict('records')
        print(f"Loaded {len(comprehensive_baseline_results)} existing results for n_estimators {n_estimators}")
        
        # Create a set of completed combinations for quick lookup
        completed_combinations = set()
        for result in comprehensive_baseline_results:
            completed_combinations.add((result['sliding_window'], result['lag_price_window']))
        
    except FileNotFoundError:
        print(f"No existing results for n_estimators {n_estimators}. Starting fresh...")
        comprehensive_baseline_results = []
        completed_combinations = set()

    # Set the number of estimators for the model
    model.set_params(n_estimators=n_estimators)
    print(f"Running Random Forest with n_estimators: {n_estimators}")

    for sliding_window in sliding_windows:
        for lag_price_window in lag_price_windows:
            # Skip if this combination has already been completed
            if (sliding_window, lag_price_window) in completed_combinations:
                print(f"Skipping already completed combination: sliding_window={sliding_window}, lag_price_window={lag_price_window}")
                continue
                
            print(f"\nRunning regression training with sliding window: {sliding_window}, price feature window: {lag_price_window}")

            # Run the regression training
            prediction_df = random_forest_training_gt(df, model, sliding_window, lag_price_window, DEBUG)

            # Calculate overall metrics
            actuals_list = prediction_df['Actual'].values
            predictions_list = prediction_df['Predicted'].values
            timestamps_list = prediction_df['Timestamp'].values

            mse = mean_squared_error(actuals_list, predictions_list)
            mae = mean_absolute_error(actuals_list, predictions_list)
            r2 = r2_score(actuals_list, predictions_list)

            # Calculate ES-like metric
            worst_overall_error_threshold = np.percentile(np.abs(predictions_list - actuals_list), 5)
            worst_overall_indices = np.abs(predictions_list - actuals_list) >= worst_overall_error_threshold
            avg_worst_error = np.mean(np.abs(predictions_list[worst_overall_indices] - actuals_list[worst_overall_indices]))

            # Calculate prediction errors for filtering
            prediction_errors = np.abs(np.array(predictions_list) - np.array(actuals_list))

            # Store results for this combination
            combination_results = []

            # Add overall results (100% percentile)
            combination_results.append({
                'sliding_window': sliding_window,
                'lag_price_window': lag_price_window,
                'percentile': 100,
                'data_points': len(predictions_list),
                'mse': mse,
                'mae': mae,
                'r2': r2,
                'worst_avg_error': avg_worst_error
            })

            for p in percentiles:
                # Filter by best predictions (lowest errors)
                best_error_threshold = np.percentile(prediction_errors, p)
                worst_error_threshold = np.percentile(prediction_errors, 100 - p)

                # Get indices of best/worst predictions 
                best_indices = prediction_errors <= best_error_threshold
                worst_indices = prediction_errors >= worst_error_threshold

                filtered_actuals = np.array(actuals_list)[best_indices]
                filtered_predictions = np.array(predictions_list)[best_indices]

                if len(filtered_actuals) > 0:
                    mse_p = mean_squared_error(filtered_actuals, filtered_predictions)
                    mae_p = mean_absolute_error(filtered_actuals, filtered_predictions)
                    r2_p = r2_score(filtered_actuals, filtered_predictions)
                    avg_error_wp = np.mean(prediction_errors[worst_indices])

                    combination_results.append({
                        'sliding_window': sliding_window,
                        'lag_price_window': lag_price_window,
                        'percentile': p,
                        'data_points': len(filtered_predictions),
                        'mse': mse_p,
                        'mae': mae_p,
                        'r2': r2_p,
                        'worst_avg_error': avg_error_wp
                    })
                else:
                    combination_results.append({
                        'sliding_window': sliding_window,
                        'lag_price_window': lag_price_window,
                        'percentile': p,
                        'data_points': 0,
                        'mse': np.nan,
                        'mae': np.nan,
                        'r2': np.nan,
                        'worst_avg_error': np.nan
                    })

            # Add this combination's results to the comprehensive list
            comprehensive_baseline_results.extend(combination_results)
            
            # Save results immediately after each combination
            results_df = pd.DataFrame(comprehensive_baseline_results)
            results_df.to_csv(results_file, index=False)
            print(f"Saved results for sliding_window={sliding_window}, lag_price_window={lag_price_window}")
            
            # Add to completed combinations
            completed_combinations.add((sliding_window, lag_price_window))

    print(f"Completed all combinations for n_estimators={n_estimators}")

Loaded 15 existing results for n_estimators 100
Running Random Forest with n_estimators: 100
Skipping already completed combination: sliding_window=7, lag_price_window=1
Skipping already completed combination: sliding_window=7, lag_price_window=2
Skipping already completed combination: sliding_window=7, lag_price_window=4

Running regression training with sliding window: 7, price feature window: 6
Number of models to train: 2252


KeyboardInterrupt: 

In [5]:
# Debug parameter
DEBUG = False

# Define the Sliding Windows for the runs
sliding_windows = [7, 10, 15, 30, 90]  # of days to train on (matrix rows)
lag_price_windows = [1, 2, 4, 6]  # of previous days to use as features (matrix columns)
# lag_price_window must be less than sliding_window
percentiles = [99, 95, 85, 75]

# Random Forest hyperparameters
n_estimators = [100, 200, 300, 400]  # Number of trees in the Random Forest
# Start with a moderately high number (e.g., 100-200) and increase it until the validation error no longer significantly improves.

max_depth = [None, 10, 20, 30]  # Maximum depth of the trees
# While default None often works well for Random Forests due to the ensemble nature
# you might experiment with values like 10, 20, or 30
# to control complexity if you observe overfitting.

# Random Forest model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=42)
# Set n_jobs=-1 to use all available cores for parallel processing
# Set random_state for reproducibility

import time
import gc
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import threading

def process_single_combination(args):
    """Process a single (sliding_window, lag_price_window, n_estimators, max_depth) combination"""
    df, model_params, sliding_window, lag_price_window, percentiles, DEBUG = args
    
    # Create model instance for this process/thread
    model = RandomForestRegressor(**model_params)
    
    print(f"Processing sliding_window={sliding_window}, lag_price_window={lag_price_window}, "
          f"n_estimators={model_params['n_estimators']}, max_depth={model_params['max_depth']}")
    
    try:
        # Run the training
        prediction_df = random_forest_training_gt(df, model, sliding_window, lag_price_window, DEBUG)
        
        # Calculate metrics
        actuals_list = prediction_df['Actual'].values
        predictions_list = prediction_df['Predicted'].values
        
        mse = mean_squared_error(actuals_list, predictions_list)
        mae = mean_absolute_error(actuals_list, predictions_list)
        r2 = r2_score(actuals_list, predictions_list)
        
        # Calculate ES-like metric
        worst_overall_error_threshold = np.percentile(np.abs(predictions_list - actuals_list), 5)
        worst_overall_indices = np.abs(predictions_list - actuals_list) >= worst_overall_error_threshold
        avg_worst_error = np.mean(np.abs(predictions_list[worst_overall_indices] - actuals_list[worst_overall_indices]))
        
        # Calculate prediction errors for filtering
        prediction_errors = np.abs(np.array(predictions_list) - np.array(actuals_list))
        
        # Store results for this combination
        combination_results = []
        
        # Add overall results (100% percentile)
        combination_results.append({
            'sliding_window': sliding_window,
            'lag_price_window': lag_price_window,
            'n_estimators': model_params['n_estimators'],
            'max_depth': model_params['max_depth'],
            'percentile': 100,
            'data_points': len(predictions_list),
            'mse': mse,
            'mae': mae,
            'r2': r2,
            'worst_avg_error': avg_worst_error
        })
        
        for p in percentiles:
            # Filter by best predictions (lowest errors)
            best_error_threshold = np.percentile(prediction_errors, p)
            worst_error_threshold = np.percentile(prediction_errors, 100 - p)
            
            # Get indices of best/worst predictions 
            best_indices = prediction_errors <= best_error_threshold
            worst_indices = prediction_errors >= worst_error_threshold
            
            filtered_actuals = np.array(actuals_list)[best_indices]
            filtered_predictions = np.array(predictions_list)[best_indices]
            
            if len(filtered_actuals) > 0:
                mse_p = mean_squared_error(filtered_actuals, filtered_predictions)
                mae_p = mean_absolute_error(filtered_actuals, filtered_predictions)
                r2_p = r2_score(filtered_actuals, filtered_predictions)
                avg_error_wp = np.mean(prediction_errors[worst_indices])
                
                combination_results.append({
                    'sliding_window': sliding_window,
                    'lag_price_window': lag_price_window,
                    'n_estimators': model_params['n_estimators'],
                    'max_depth': model_params['max_depth'],
                    'percentile': p,
                    'data_points': len(filtered_predictions),
                    'mse': mse_p,
                    'mae': mae_p,
                    'r2': r2_p,
                    'worst_avg_error': avg_error_wp
                })
            else:
                combination_results.append({
                    'sliding_window': sliding_window,
                    'lag_price_window': lag_price_window,
                    'n_estimators': model_params['n_estimators'],
                    'max_depth': model_params['max_depth'],
                    'percentile': p,
                    'data_points': 0,
                    'mse': np.nan,
                    'mae': np.nan,
                    'r2': np.nan,
                    'worst_avg_error': np.nan
                })
        
        # Memory cleanup
        del prediction_df, actuals_list, predictions_list, prediction_errors
        gc.collect()
        
        return combination_results
        
    except Exception as e:
        print(f"Error processing combination: {e}")
        return []

# Debug parameter
DEBUG = False

# Define the Sliding Windows for the runs
sliding_windows = [7, 10, 15, 30, 90]
lag_price_windows = [1, 2, 4, 6]
percentiles = [99, 95, 85, 75]

# Random Forest hyperparameters (including max_depth)
n_estimators = [100, 200, 300, 400]
max_depth = [None, 10, 20, 30]

# Get the number of threads for optimal parallelization
# Use fewer threads than CPU cores to avoid overwhelming the system
import os
max_workers = min(os.cpu_count(), 4)  # Limit to 4 threads max for stability
print(f"Using {max_workers} parallel threads")

# Process all combinations with parallel processing using Threading
for n_est in n_estimators:
    for max_d in max_depth:
        # Initialize results file path
        results_file = f"random_forest_baseline_results_n_estimators_{n_est}_max_depth_{max_d}.csv"
        
        # Check if results file already exists and load existing results
        try:
            existing_results_df = pd.read_csv(results_file)
            comprehensive_baseline_results = existing_results_df.to_dict('records')
            print(f"Loaded {len(comprehensive_baseline_results)} existing results for n_estimators={n_est}, max_depth={max_d}")
            
            # Create a set of completed combinations for quick lookup
            completed_combinations = set()
            for result in comprehensive_baseline_results:
                completed_combinations.add((result['sliding_window'], result['lag_price_window']))
            
        except FileNotFoundError:
            print(f"No existing results for n_estimators={n_est}, max_depth={max_d}. Starting fresh...")
            comprehensive_baseline_results = []
            completed_combinations = set()
        
        # Prepare model parameters
        model_params = {
            'n_estimators': n_est,
            'max_depth': max_d,
            'n_jobs': -1,  # Let each RandomForest use all cores since we're using threading
            'random_state': 42,
            'warm_start': False,  # Disable warm_start for threading
            'max_samples': 0.8,
            'max_features': 'sqrt',
            'min_samples_split': 5,
            'min_samples_leaf': 2
        }
        
        # Prepare all combinations that haven't been completed
        combinations = []
        for sliding_window in sliding_windows:
            for lag_price_window in lag_price_windows:
                if (sliding_window, lag_price_window) not in completed_combinations:
                    combinations.append((df, model_params, sliding_window, lag_price_window, percentiles, DEBUG))
        
        if not combinations:
            print(f"All combinations already completed for n_estimators={n_est}, max_depth={max_d}")
            continue
        
        print(f"Processing {len(combinations)} combinations for n_estimators={n_est}, max_depth={max_d}")
        
        # Progress monitoring
        start_time = time.time()
        
        # Process sequentially but with progress monitoring for better stability
        with tqdm(total=len(combinations), desc=f"RF n_est={n_est}, max_depth={max_d}") as pbar:
            for i, combination in enumerate(combinations):
                # Process single combination
                result = process_single_combination(combination)
                
                if result:  # Check if result is not empty
                    comprehensive_baseline_results.extend(result)
                    
                    # Update completed combinations
                    sliding_window, lag_price_window = combination[2], combination[3]
                    completed_combinations.add((sliding_window, lag_price_window))
                    
                    # Save results after each combination
                    results_df = pd.DataFrame(comprehensive_baseline_results)
                    results_df.to_csv(results_file, index=False)
                
                # Update progress
                pbar.update(1)
                
                # Calculate and display timing information
                elapsed = time.time() - start_time
                if i > 0:
                    avg_time_per_combo = elapsed / (i + 1)
                    remaining_combos = len(combinations) - (i + 1)
                    remaining_time = avg_time_per_combo * remaining_combos
                    
                    pbar.set_postfix({
                        'ETA': f"{remaining_time/3600:.1f}h",
                        'Avg': f"{avg_time_per_combo:.1f}s/combo"
                    })
                
                # Memory cleanup
                gc.collect()
        
        print(f"Completed all combinations for n_estimators={n_est}, max_depth={max_d}")
        print(f"Total time: {(time.time() - start_time)/3600:.2f} hours")

# Final comprehensive results aggregation
print("\nAggregating all results...")
comprehensive_gt_df = pd.DataFrame()

for n_est in n_estimators:
    for max_d in max_depth:
        try:
            results_file = f"random_forest_baseline_results_n_estimators_{n_est}_max_depth_{max_d}.csv"
            results_df = pd.read_csv(results_file)
            comprehensive_gt_df = pd.concat([comprehensive_gt_df, results_df], ignore_index=True)
            print(f"Added results for n_estimators={n_est}, max_depth={max_d}")
        except FileNotFoundError:
            print(f"Results file not found for n_estimators={n_est}, max_depth={max_d}")

# Save comprehensive results
comprehensive_gt_df.to_csv('random_forest_full_baseline_results_optimized.csv', index=False)
print(f"\nFinal results saved to 'random_forest_full_baseline_results_optimized.csv'")
print(f"Total combinations processed: {len(comprehensive_gt_df)}")

# Memory cleanup
gc.collect()

Using 4 parallel threads
No existing results for n_estimators=100, max_depth=None. Starting fresh...
Processing 20 combinations for n_estimators=100, max_depth=None


RF n_est=100, max_depth=None:   0%|          | 0/20 [00:00<?, ?it/s]

Processing sliding_window=7, lag_price_window=1, n_estimators=100, max_depth=None
Number of models to train: 2252


RF n_est=100, max_depth=None:   5%|▌         | 1/20 [02:38<50:09, 158.38s/it]

Processing sliding_window=7, lag_price_window=2, n_estimators=100, max_depth=None
Number of models to train: 2252


RF n_est=100, max_depth=None:  10%|█         | 2/20 [05:21<48:18, 161.05s/it, ETA=0.8h, Avg=160.6s/combo]

Processing sliding_window=7, lag_price_window=4, n_estimators=100, max_depth=None
Number of models to train: 2252


RF n_est=100, max_depth=None:  10%|█         | 2/20 [05:24<48:39, 162.18s/it, ETA=0.8h, Avg=160.6s/combo]


KeyboardInterrupt: 

In [13]:
# Import all results to one single DataFrame
comprehensive_gt_df = pd.DataFrame()

n_estimators = [100, 200, 300, 400]
max_depths = [10, 20, 30, 999]

# Add up all results for different configurations of n_estimators
for n_estimators in n_estimators:
    for max_depth in max_depths:
        try:
            results_df = pd.read_csv(f"rf_results/random_forest_baseline_results_n_estimators_{n_estimators}_max_depth_{max_depth}.csv")
            results_df['n_estimators'] = n_estimators  # Add n_estimators column
            results_df['max_depth'] = max_depth  # Add max_depth column
            comprehensive_gt_df = pd.concat([comprehensive_gt_df, results_df], ignore_index=True)
        except FileNotFoundError:
            print(f"Results file not found for n_estimators={n_estimators}, max_depth={max_depth}")
# Save comprehensive results
comprehensive_gt_df.to_csv('rf_results/random_forest_full_baseline_results_optimized.csv', index=False)
print(f"\nFinal results saved to 'rf_results/random_forest_full_baseline_results_optimized.csv'")
print(f"Total combinations processed: {len(comprehensive_gt_df)}")

Results file not found for n_estimators=400, max_depth=20
Results file not found for n_estimators=400, max_depth=30

Final results saved to 'rf_results/random_forest_full_baseline_results_optimized.csv'
Total combinations processed: 1400


In [16]:
# Pareto frontier approach - find models that are best in multiple metrics
def find_pareto_optimal_models(df, percentile=99):
    """Find models that are optimal in at least one metric"""
    subset = df[df['percentile'] == percentile].copy()
    
    # Rank by each metric (lower rank = better)
    subset['mae_rank'] = subset['mae'].rank(ascending=True)
    subset['mse_rank'] = subset['mse'].rank(ascending=True)  # Lower MSE is better
    subset['r2_rank'] = subset['r2'].rank(ascending=False)  # Higher R² is better
    subset['worst_error_rank'] = subset['worst_avg_error'].rank(ascending=True)
    
    # Find average rank across all 4 metrics
    subset['avg_rank'] = (subset['mae_rank'] + subset['mse_rank'] + 
                         subset['r2_rank'] + subset['worst_error_rank']) / 4
    
    return subset.sort_values('avg_rank')

# Get top models by multiple criteria
pareto_gt = find_pareto_optimal_models(comprehensive_gt_df)
# pareto_fe = find_pareto_optimal_models(comprehensive_fe_df)

print("Top 3 models by multi-criteria ranking (including MSE):")
print(pareto_gt[['sliding_window', 'lag_price_window', 'n_estimators', 'max_depth', 'mae', 'mse', 'r2', 'worst_avg_error', 'avg_rank']].head(3))

Top 3 models by multi-criteria ranking (including MSE):
     sliding_window  lag_price_window  n_estimators  max_depth       mae  \
516               7                 6           200         20  15.06236   
616               7                 6           200         30  15.06236   
716               7                 6           200        999  15.06236   

            mse        r2  worst_avg_error  avg_rank  
516  592.975588  0.819059          16.4993     3.375  
616  592.975588  0.819059          16.4993     3.375  
716  592.975588  0.819059          16.4993     3.375  


In [ ]:
# recalculate the best model predictions
print(f"Best Baseline Model - Sliding Window: {30}, Lag Price Window: {1}")

best_baseline_model = 

# Run the lasso training for the best model
prediction_df_best_baseline = lasso_training_gt(df, best_baseline_model, 30, 1, DEBUG=False)

# Calculate prediction errors from the current model run
current_actuals = prediction_df_best_baseline['Actual'].values
current_predictions = prediction_df_best_baseline['Predicted'].values
current_prediction_errors = np.abs(current_predictions - current_actuals)

# Limit results to the 99th percentile using CURRENT prediction errors
best_error_threshold = np.percentile(current_prediction_errors, 99)  # Use current_prediction_errors
# Get indices of best predictions using CURRENT prediction errors
best_indices = current_prediction_errors <= best_error_threshold  # Use current_prediction_errors

# Filter the DataFrame directly
prediction_df_best_baseline = prediction_df_best_baseline[best_indices].reset_index(drop=True)

In [ ]:
import matplotlib.pyplot as plt
# Plotting the predictions for the best model as scatter plot over the actuals as line plot
plt.figure(figsize=(14, 7))
plt.plot(prediction_df_best_baseline['Timestamp'], prediction_df_best_baseline['Actual'], label='Actual', color='darkblue')
plt.scatter(prediction_df_best_baseline['Timestamp'], prediction_df_best_baseline['Predicted'], label='Predicted', color='orange')
plt.title(f"Predicted vs Actual Prices\nSliding Window: {int(30)}, Lag Price Window: {int(1)}")
plt.xlabel('Timestamp')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('lasso_baseline_best_model.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# plot r2
 


Feature engineering version:

In [ ]:
alphas = [0.01, 0.1, 1, 10]  # Lasso regularization parameters

from sklearn.linear_model import Lasso
# Initialize the regression model
model = Lasso()

for alpha in alphas:
    # Initialize comprehensive results list
    comprehensive_fe_results = []

    # Set the alpha parameter for Lasso
    model.set_params(alpha=alpha)
    print(f"\nRunning regression training with Lasso alpha: {alpha}")

    for sliding_window in sliding_windows:
        for lag_price_window in lag_price_windows:
            print(f"\nRunning regression training with sliding window: {sliding_window}, price feature window: {lag_price_window}")

            # Run the regression training
            prediction_df = regression_training(df, sliding_window, lag_price_window, DEBUG)
            
            # Calculate overall metrics
            actuals_list = prediction_df['Actual'].values
            predictions_list = prediction_df['Predicted'].values
            timestamps_list = prediction_df['Timestamp'].values

            mse = mean_squared_error(actuals_list, predictions_list)
            mae = mean_absolute_error(actuals_list, predictions_list)
            r2 = r2_score(actuals_list, predictions_list)
            
            # Expectation Shortfall - Calculate metrics for top X worst predictions
            # Calculate prediction errors for filtering
            prediction_errors = np.abs(np.array(predictions_list) - np.array(actuals_list))

            # Add overall results (100% percentile)
            comprehensive_fe_results.append({
                'sliding_window': sliding_window,
                'lag_price_window': lag_price_window,
                'percentile': 100,
                'data_points': len(predictions_list),
                'mse': mse,
                'mae': mae,
                'r2': r2,
                'avg_error': np.mean(prediction_errors)
            })
            
            for p in percentiles:
                # Filter by best predictions (lowest errors)
                best_error_threshold = np.percentile(prediction_errors, p)
                
                # Expectation Shortfall like metric - worst predictions analysis
                # Filter by best predictions (highest errors)
                worst_error_threshold = np.percentile(prediction_errors, 100 - p) # equivalent to worst_percentiles

                # Get indices of best/worst predictions 
                best_indices = prediction_errors <= best_error_threshold # with errors below threshold
                worst_indices = prediction_errors >= worst_error_threshold # with errors above threshold

                filtered_actuals = np.array(actuals_list)[best_indices]
                filtered_predictions = np.array(predictions_list)[best_indices]

                if len(filtered_actuals) > 0:
                    mse_p = mean_squared_error(filtered_actuals, filtered_predictions)
                    mae_p = mean_absolute_error(filtered_actuals, filtered_predictions)
                    r2_p = r2_score(filtered_actuals, filtered_predictions)
                    avg_error_wp = np.mean(prediction_errors[worst_indices])

                    # Add to full comprehensive results
                    comprehensive_fe_results.append({
                        'sliding_window': sliding_window,
                        'lag_price_window': lag_price_window,
                        'percentile': p,
                        'data_points': len(filtered_predictions),
                        'mse': mse_p,
                        'mae': mae_p,
                        'r2': r2_p,
                        'worst_avg_error': avg_error_wp
                    })
                else:
                    # Add entry for no data available
                    comprehensive_fe_results.append({
                        'sliding_window': sliding_window,
                        'lag_price_window': lag_price_window,
                        'percentile': p,
                        'data_points': 0,
                        'mse': np.nan,
                        'mae': np.nan,
                        'r2': np.nan,
                        'worst_avg_error': np.nan
                    })
    # Save results to CSV
    results_df = pd.DataFrame(comprehensive_fe_results)
    results_df.to_csv(f"lasso_fe_results_alpha_{alpha}.csv", index=False)

In [ ]:
# Create comprehensive DataFrame
comprehensive_fe_df = pd.DataFrame(comprehensive_fe_results)

# Save to CSV
comprehensive_fe_df.to_csv('regression_fe_results.csv', index=False)
print(f"\nResults saved to 'regression_fe_results.csv'")

In [ ]:
# Determine the best performing model based on lowest MAE and 99th percentile
best_fe_model = comprehensive_fe_df.loc[comprehensive_fe_df['percentile'] == 99].sort_values(by='mae').iloc[0]
print("\nBest performing model based on lowest MAE:")
print(best_fe_model)

In [ ]:
# Pareto frontier approach - find models that are best in multiple metrics
def find_pareto_optimal_models(df, percentile=99):
    """Find models that are optimal in at least one metric"""
    subset = df[df['percentile'] == percentile].copy()
    
    # Rank by each metric (lower rank = better)
    subset['mae_rank'] = subset['mae'].rank(ascending=True)
    subset['mse_rank'] = subset['mse'].rank(ascending=True)  # Lower MSE is better
    subset['r2_rank'] = subset['r2'].rank(ascending=False)  # Higher R² is better
    subset['worst_error_rank'] = subset['worst_avg_error'].rank(ascending=True)
    
    # Find average rank across all 4 metrics
    subset['avg_rank'] = (subset['mae_rank'] + subset['mse_rank'] + 
                         subset['r2_rank'] + subset['worst_error_rank']) / 4
    
    return subset.sort_values('avg_rank')

# Get top models by multiple criteria
pareto_fe = find_pareto_optimal_models(comprehensive_fe_df)

print("Top 3 models by multi-criteria ranking (including MSE):")
print(pareto_fe[['sliding_window', 'lag_price_window', 'mae', 'mse', 'r2', 'worst_avg_error', 'avg_rank']].head(3))

In [ ]:
# recalculate the best model predictions
best_fe_sliding_window = best_fe_model['sliding_window']
best_fe_lag_price_window = best_fe_model['lag_price_window']

print(f"Best Feature Engineering Model - Sliding Window: {best_fe_sliding_window}, Lag Price Window: {best_fe_lag_price_window}")

# Run the regression training for the best model
prediction_df_best_fe = regression_training(df, int(best_fe_sliding_window), int(best_fe_lag_price_window), DEBUG=False)

In [ ]:
import matplotlib.pyplot as plt
# Plotting the predictions for the best model as scatter plot over the actuals as line plot
plt.figure(figsize=(14, 7))
plt.plot(prediction_df_best_fe['Timestamp'], prediction_df_best_fe['Actual'], label='Actual', color='darkblue')
plt.scatter(prediction_df_best_fe['Timestamp'], prediction_df_best_fe['Predicted'], label='Predicted', color='orange')
plt.title(f"Predicted vs Actual Prices\nSliding Window: {int(best_fe_model['sliding_window'])}, Lag Price Window: {int(best_fe_model['lag_price_window'])}")
plt.xlabel('Timestamp')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('regression_fe_best_model.png', dpi=300, bbox_inches='tight')
plt.show()